In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

In [ ]:
rows_1=32
cols_1=17
rows_2=30
cols_2=32
epsilon=0.00000
gamma=1
reward_step=-1
action=[[-1,-1],[-1,0],[-1,1],[0,-1],[0,0],[0,1],[1,-1],[1,0],[1,1]]
velocity_limit=5
Q=np.zeros((rows_1,cols_1,velocity_limit,velocity_limit,len(action)))
policy1=np.zeros((rows_1,cols_1,velocity_limit,velocity_limit),dtype=int)
policy2=np.zeros((rows_1,cols_1,velocity_limit,velocity_limit),dtype=int)
policy_1={}
policy_2={}
for i in range(rows_1):
    for j in range(cols_1):
        for m in range(velocity_limit):
            for n in range(velocity_limit):
                policy_1[i,j,m,n]=[policy1[i,j,m,n],policy2[i,j,m,n]]
                p=[]
                for k in range(len(action)):
                    p.append(Q[i,j,m,n,k])         
                policy_1[i,j,m,n]=action[np.argmax(p)]
C=np.zeros(Q.shape,dtype=float)
b=np.zeros(Q.shape,dtype=int)
state={}
A={}
B={}

In [ ]:
race_track1=np.zeros((rows_1,cols_1),dtype=int)
race_track1[0,3:]=1
race_track1[0,3:]=-10  #boundary
race_track1[1:3,2:]=1
race_track1[1:3,2]=-10
race_track1[3,1:]=1
race_track1[3,1]=-10
race_track1[4:6,]=1
race_track1[4:6,0]=-10
race_track1[5,10:]=-10
race_track1[6,:10]=1
race_track1[6,0]=-10
race_track1[6,9]=-10
race_track1[7:14,:9]=1
race_track1[7:14,0]=-10
race_track1[14:22,1:9]=1
race_track1[14:22,1]=-10
race_track1[22:29,2:9]=1
race_track1[22:29,2]=-10
race_track1[29:,3:9]=1
race_track1[29:32,3]=-10
race_track1[7:,8]=-10
race_track1[31,3:9]=50
start=race_track1[31,3:9]
race_track1[0:6,16]=-50
finish=race_track1[0:6,16]

In [ ]:
race_track2=np.zeros((rows_2,cols_2),dtype=int)
race_track2[0,16:]=-10
race_track2[1,13:]=1
race_track2[1,13:17]=-10
race_track2[2,12:]=1
race_track2[2,12]=-10
race_track2[3:7,11:]=1
race_track2[3:7,11]=-10
race_track2[7,12:]=1
race_track2[7,12]=-10
race_track2[8,13:]=1
race_track2[8,13]=-10
race_track2[9,14:30]=1
race_track2[8,30:]=-10
race_track2[9,27:30]=-10
race_track2[10,14:27]=1
race_track2[10,26]=-10
race_track2[11,14:26]=1
race_track2[11,24:26]=-10
race_track2[12,14:24]=1
race_track2[12,23]=-10
race_track2[13,14:23]=1
race_track2[9:14,14]=-10
race_track2[14,13:23]=1
race_track2[15,12:23]=1
race_track2[16,11:23]=1
race_track2[17,10:23]=1
race_track2[18,9:23]=1
race_track2[19,8:23]=1
race_track2[20,7:23]=1
race_track2[21,6:23]=1
race_track2[22,5:23]=1
race_track2[23,4:23]=1
race_track2[24,3:23]=1
race_track2[25,2:23]=1
race_track2[26,1:23]=1
race_track2[27:,:23]=1
race_track2[27:,0]=-10
race_track2[13:,22]=-10
for i in range(1,14):
    race_track2[13+i,14-i]=-10
race_track2[29,:23]=50
race_track2[0:9,31]=-50
# plt.figure()
# plt.matshow(race_track2)

In [ ]:
def valid_action(t,state):
    valid=[]
    for i in range(-velocity_limit+1,velocity_limit):
        for j in range(-velocity_limit+1,velocity_limit):
            if 0<=i+state[t][2]<=4 and 0<=j+state[t][3]<=4 and not (i+state[t][2]==0 and j+state[t][3]==0):
                if -1<=i<=1 and -1<=j<=1:
                    valid.append([i,j])
    return valid

In [ ]:
def policy_maker1():
    t=0
    state[t]=[]
    start_row=31
    start_col=np.random.randint(3,9)
    start_vel_row=1
    start_vel_col=1
    state[t].append(start_row)
    state[t].append(start_col)
    state[t].append(start_vel_row)
    state[t].append(start_vel_col)
    while True:
        s=state[t]
        c=[]
        val_act=valid_action(t,state)
        for i in range (0,len(val_act)):
            c.append(i)
        pr=[]  
        for i in range(len(val_act)):
            pr.append(1/len(val_act))
        act_num=len(val_act)
        pi=policy_1[state[t][0],state[t][1],state[t][2],state[t][3]]
        if np.random.random()>=epsilon:
            if pi in val_act:
                a=pi
                b=1-epsilon+epsilon/act_num
            else:
                a=val_act[(np.random.choice(c, p=pr))]
                b=1/act_num
        else:
            a=val_act[(np.random.choice(c, p=pr))]
            if pi in val_act:
                b=epsilon/act_num
            else:
                b=1/act_num
        A[t]=a
        B[t]=b
        act_choose=a
        vel_next=[state[t][2]+act_choose[0],state[t][3]+act_choose[1]]
        if 0<=state[t][0]-vel_next[1]-1<rows_1 and 0<=state[t][1]+vel_next[0]-1<17 and not(vel_next[0]==0 and vel_next[1]==0) and 0<=vel_next[0]<=4 and 0<=vel_next[1]<=4:
            next=[state[t][0]-vel_next[1]-1,state[t][1]+vel_next[0]-1,vel_next[0],vel_next[1]]
            if race_track1[next[0],next[1]]==1:
                state_next=next
                s=state_next
            elif race_track1[next[0],next[1]]==-50:
                return t,state
            elif race_track1[next[0],next[1]]==-10:
                start_col=np.random.randint(3,9)
                state_next=[start_row,start_col,start_vel_row,start_vel_col]
                s=state_next
            elif race_track1[next[0],next[1]]==0:
                start_col=np.random.randint(3,9)
                state_next=[start_row,start_col,start_vel_row,start_vel_col]
                s=state_next
            else:
                state_next=s
                s=state_next
        else:
            state_next=[state[t][0],state[t][1],1,1]
            s=state_next
        t+=1
        state[t]=[]
        state[t].append(s[0])
        state[t].append(s[1])
        state[t].append(s[2])
        state[t].append(s[3])
    
    

In [ ]:
def for_each_episode(T,state):
    G=0.0
    W=1.0
    x=[]
    y=[]
    for t in range(T-1,-1,-1):
        G=gamma*G+reward_step
        C[state[t][0],state[t][1],state[t][2],state[t][3],A[t]]+=W
        Q[state[t][0],state[t][1],state[t][2],state[t][3],A[t]]+=W*(G-Q[state[t][0],state[t][1],state[t][2],state[t][3],A[t]])/C[state[t][0],state[t][1],state[t][2],state[t][3],A[t]]
        val_act=valid_action(t,state)
        policy_1[state[t][0],state[t][1],state[t][2],state[t][3]]=val_act[np.argmax(Q[state[t][0],state[t][1],state[t][2],state[t][3],:])]
        x.append(state[t][0])
        y.append(state[t][1])
        if A[t]==policy_1[state[t][0],state[t][1],state[t][2],state[t][3]]:
            return t,x,y
        W/=B[t]
        # return t,x,y


In [ ]:
def track_para():
    for i in range(2):
        T,state_update=policy_maker1()
        print("trace:")
        for j in range(T):
            print("S:",state_update[j])
            print("A:",A[j])
        print("R:",reward_step*T)


In [ ]:
if __name__ == '__main__':
    episode_num=20
    reward=[]
    plt.figure()
    plt.matshow(race_track1)
    for i in range(episode_num):
        x=[]
        y=[]
        T,state_update=policy_maker1()
        t,_,__=for_each_episode(T,state_update)
        for j in range(T-1,t-15,-1):
            x.append(state_update[j][0])
            y.append(state_update[j][1])
        print(t)
        print(x)
        print(y)
        plt.plot(np.transpose(y),np.transpose(x))
        plt.savefig('trace1.png',bbox_inches='tight')
        print("episode"+str(i)+":"+str(T)+","+str(t)+","+str(T-t))
        if i%9==0:
            T,state_update=policy_maker1()   
            reward.append(reward_step*T)
    track_para()
    plt.show()
